In [71]:
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup
import requests
import pymongo
import datetime
import json
import requests
import time

# Google developer API key
# from config import api_key

## Scrape Data

In [72]:
url = 'https://www.worldometers.info/coronavirus/'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)

df = dfs[0]

df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop
0,World,1697255,"+93,561",102659.0,"+6,975",376106.0,1218490,49830.0,218.00,13.2,NaN,NaN
1,USA,501821,"+33,255",18695.0,"+2,013",27239.0,455887,10916.0,1516.00,56.0,2536189.0,7662.0
2,Spain,158273,"+5,051",16081.0,+634,55668.0,86524,7371.0,3385.00,344.0,355000.0,7593.0
3,Italy,147577,"+3,951",18849.0,+570,30455.0,98273,3497.0,2441.00,312.0,906864.0,14999.0
4,France,124869,"+7,120",13197.0,+987,24932.0,86740,7004.0,1913.00,202.0,333807.0,5114.0
...,...,...,...,...,...,...,...,...,...,...,...,...
209,Timor-Leste,2,+1,NaN,NaN,1.0,1,NaN,2.00,NaN,NaN,NaN
210,Saint Pierre Miquelon,1,NaN,NaN,NaN,NaN,1,NaN,173.00,NaN,NaN,NaN
211,Yemen,1,+1,NaN,NaN,NaN,1,NaN,0.03,NaN,NaN,NaN
212,China,81907,NaN,3336.0,NaN,77455.0,1116,144.0,57.00,2.0,NaN,NaN


## Coords

In [ ]:
countriesScraped = list(df['Country,Other'])
# countries
# The default port used by MongoDB is 27017
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.corona_db
collection = db.corona_data
results = db.corona_data.find()[0]
existingCountries = list(results['Country,Other'].values())

# for country in countriesScraped:
#     if country not in existingCountries:
#         print(country)
#         Run api call to get coords


countries = list(df['Country,Other'])
countryCount = 0
coordinates = []
for country in countries:
#     print(country)
    url = (f'https://maps.googleapis.com/maps/api/geocode/json?address={country}&key={api_key}')
    # fix this with not in []
    if country not in ['World','Total:']:
        print(country)
        try:
            response = requests.get(url).json()
            result = response['results'][0]['geometry']['location']
            loc = [result['lat'],result['lng']]

            print(loc)
        except:
            print('Failed Request')
            loc = [0,0]
        coordinates.append(loc)
        time.sleep(3)
    else:
        loc = [0,0]
        coordinates.append(loc)
    
    
    countryCount +=1 


In [ ]:
df.to_json('countries2.json')

In [ ]:
# Insert document into collection
with open('countries2.json') as f:
    countries_data = json.load(f)



In [ ]:
countries_data

In [ ]:
collection.insert_one(countries_data)

## Database coords

In [ ]:
# Insert document into collection
with open('countries.json') as f:
    countries_data = json.load(f)

In [ ]:
data = pd.read_json('countries.json')

In [ ]:
data

In [ ]:
countries = data['Country,Other']
coordinates = data['Coords']

In [ ]:
coordsDF = pd.DataFrame({
    'country': countries,
    'coords': coordinates
})

In [ ]:
coordsDF

In [ ]:
coordsDF.to_json('coords.json')

In [ ]:
# Insert document into collection
with open('coords.json') as f:
    coords_data = json.load(f)

In [ ]:
# coords_data
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.corona_db
collection = db.country_coords

In [ ]:
collection.insert_one(coords_data)

In [ ]:
results = db.country_coords.find()
# for result in results:
#     print(result)
# result

In [ ]:
result['country']

## Comparison Testing

In [ ]:
df['Coords'] = ''
df.head()

In [ ]:
oldDF = pd.read_json('countries.json')
oldDF.head()

In [ ]:
newCountriesList = list(df['Country,Other'])
oldCountriesList = list(oldDF['Country,Other'])
needCoords = []

In [ ]:
for x in newCountriesList:
    if x not in oldCountriesList:
        needCoords.append(x)
        print(x)

In [ ]:
needCoords

In [ ]:
# coords_data
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.corona_db
collection = db.country_coords

In [ ]:
coordsData = collection.find_one()
countries = coordsData['country']
coordinates = coordsData['coords']

coordsDF = pd.DataFrame({
    'country': countries,
    'coords': coordinates
})

In [ ]:
coordsDF

In [ ]:
for index1,row1 in df.iterrows():
    xCountry = row1['Country,Other']
#     print(xCountry)
    query = collection.find({'country':xCountry})
    print(query)
#     for index2,row2 in coordsDF.iterrows():
#         yCountry = row2['country']
#         yCoords = row2['coords']
#         if yCountry == xCountry:
#             print(xCountry,yCoords)
#             row1['Coords'] = yCoords
            
    